<a href="https://colab.research.google.com/github/ZaneZaiontz/YosAI/blob/main/YosAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from keras.preprocessing import sequence
import keras
import numpy as np
import os
import random as rand
import sys

Open dataset


In [ ]:
# Read, then decode for py2 compat.
file = open('./zhaoClean.txt', 'r')
lines = file.readlines()
file.close()
file = open('./zhaoClean.txt', 'r')
text = file.read().lower()
# num chars in file
print ('Length of text: {} characters'.format(len(text)))

Length of text: 256636 characters


Encoding


In [ ]:
vocab = sorted(set(text))
# UID's for each char
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

Text: a skein of bi
Encoded: [25  1 43 35 29 33 38  1 39 30  1 26 33]


Step 3

In [ ]:
seq_length = 64  # training seq length, 64 ~ haiku length
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Step 4

In [ ]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

Build Model/Shuffle Data

In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000
data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Compile/Loss Function


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam', loss=loss)

Checkpoints/Fitting

In [ ]:
checkpoint_dir = './trainCKPT'
checkpoint_prefix = os.path.join(checkpoint_dir, "CKPT-{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)
history = model.fit(data, epochs=100, callbacks=[checkpoint_callback])

Build

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

Output

In [ ]:
def generate_text(model):
  newL = 0
  startInt = (rand.randint(0, 44000))/3
  startLine = lines[int(startInt)]
  startLine = startLine.lower()

  input_eval = [char2idx[s] for s in startLine]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []

  model.reset_states()
  while newL < 3:
      predictions = model(input_eval)

      predictions = tf.squeeze(predictions, 0)
      predictions = predictions
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      
      input_eval = tf.expand_dims([predicted_id], 0)

      if (idx2char[predicted_id] == '\n'):
        newL += 1
      if (newL >= 3):
        text_generated.append(idx2char[predicted_id])
        break

      # print(idx2char[predicted_id])

      text_generated.append(idx2char[predicted_id])

  return (''.join(text_generated))

print(generate_text(model))

